In [52]:
import sys

In [1]:
cd /Users/annabramslow/Documents/Company2Vec

/Users/annabramslow/Documents/Company2Vec


In [2]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import List

import dask.dataframe as dd
import pandas as pd

#from ..decorators import save_parquet
#from ..ops import sort_partitions
#from ..serialize import DATA_ROOT
#from .base import FIELD_TYPE, TokenSource
#from src.decorators import save_parquet
#from src.ops import sort_partitions
#from base import FIELD_TYPE, TokenSource
#from data_processing.helpers import enrich_with_asof_values

In [3]:
def dd_enrich_with_asof_values(
    df: dd.DataFrame, 
    df_registrations: dd.DataFrame, 
    values=['Industry', 'CompanyType', 'Address', 'Status'], 
    date_col_df='FromDate', 
    date_col_registrations='FromDate'
) -> dd.DataFrame:
    """
    Adds the as-of values of df_registrations for the entries in df.
    Allows different date column names for both dataframes.
    """

    for value in values:
        # Load data by filtering df_registrations on ChangeType
        df_value = df_registrations.loc[df_registrations['ChangeType'] == value].reset_index(drop=True)

        # Special handling for 'Status' value: replace NaN dates with '2000-01-01'
        if value == 'Status':
            df_value[date_col_registrations] = df_value[date_col_registrations].fillna('2000-01-01')

        # Convert date columns to datetime if they are not already in datetime format
        df[date_col_df] = dd.to_datetime(df[date_col_df], errors='coerce')
        df_value[date_col_registrations] = dd.to_datetime(df_value[date_col_registrations], errors='coerce')

        # Select relevant columns and rename 'NewValue' to the current value
        df_value = df_value[['CVR', date_col_registrations, 'NewValue']].rename(columns={
            date_col_registrations: date_col_df,  # Align the date column names
            'NewValue': value  # Rename 'NewValue' to the specific value being processed
        })

        # Perform the asof merge using Dask's merge_asof function
        df = dd.merge_asof(
            df.sort_values(date_col_df),
            df_value.sort_values(date_col_df),
            on=date_col_df,
            by='CVR',
            direction='backward'
        )

    return df


In [4]:
def convert_currency( df: dd.DataFrame, lookup_table: dd.DataFrame, 
                     amount_cols=['amount'],  # List of columns to convert
                     currency_col='currency', 
                     date_col='PublicationDate',  # The datetime column in df
                     ) -> dd.DataFrame:
    """
    Convert multiple currency columns in the DataFrame based on a lookup table for rows where currency is not 'DKK'.
    
    Args:
        df (dd.DataFrame): The main dataframe with amounts to convert.
        lookup_table (dd.DataFrame): The lookup table containing conversion rates.
        amount_cols (list): List of column names in df containing amounts to be converted.
        currency_col (str): Column name in df containing currency information.
        date_col (str): Column name for datetime in df to extract year and month from.
    
    Returns:
        dd.DataFrame: DataFrame with currency conversion applied to the specified amount columns.
    """

    # 1. Filter rows where the currency is not 'DKK'
    non_dkk_df = df[df[currency_col] != 'DKK']
    dkk_df = df[df[currency_col] == 'DKK']  # Keep these rows unchanged

    # 2. Extract year and month from the PublicationDate column
    non_dkk_df['year'] = non_dkk_df[date_col].dt.year
    non_dkk_df['month'] = non_dkk_df[date_col].dt.month

    # 3. Perform a join to find the correct rate for each non-DKK row
    merged_df = dd.merge(
        non_dkk_df,
        lookup_table,
        left_on=[currency_col, 'year', 'month'],
        right_on=['from_currency', 'year', 'month'],
        how='left'
    )

    # 4. Apply currency conversion for each amount column
    for col in amount_cols:
        merged_df[col] = merged_df[col] * merged_df['rate']

    # 5. Drop unnecessary columns (from the lookup table like 'rate', 'from_currency')
    merged_df = merged_df.drop(columns=['rate', 'from_currency', 'year', 'month'])

    # 6. Concatenate the DKK and non-DKK dataframes
    final_df = dd.concat([dkk_df, merged_df])

    return final_df

In [5]:
DATA_ROOT = Path.home() / "Library" / "CloudStorage" / "Dropbox" / "DTU" / "Virk2Vec" / "Tables"
path_financials = DATA_ROOT / "Financials"
path_registrations = DATA_ROOT / "Registrations"
path_currency = DATA_ROOT / "Currency"

In [6]:
#load files from the path_financials folder

# Example: Get all .txt files in the folder
financials_csv = [file for file in path_financials.iterdir() if file.is_file() and file.suffix == '.csv']
registrations_csv = [file for file in path_registrations.iterdir() if file.is_file() and file.suffix == '.csv']
currency_csv = [file for file in path_currency.iterdir() if file.is_file() and file.suffix == '.csv']


In [7]:
columns_registrations = [
    "CVR",
    "FromDate",
    "ChangeType",
    "NewValue"
]

columns_annualreport = [
    "CVR",
    "PublicationDate",
    "Currency",
    "ProfitLoss",
    "Equity", 
    "Assets", 
    "LiabilitiesAndEquity"
]

columns_currency = [
    "year",
    "month",
    "from_currency",
    "rate"
]

In [21]:
ddf_registrations = dd.read_csv(
    registrations_csv,
    usecols=columns_registrations,
    on_bad_lines="error",
    assume_missing=True,
    dtype={
        "CVR": int,
        "FromDate": str,
        "ChangeType": str,
        "NewValue": str
    },
    blocksize="256MB")

In [22]:
ddf_annualreport = dd.read_csv(
    financials_csv,
    usecols=columns_annualreport,
    on_bad_lines="error",
    assume_missing=True,
    dtype={
        "CVR": int,
        "PublicationDate": str,
        "Currency": str,
        "ProfitLoss": float,  # Deal with missing values
        "Equity": float,
        "Assets": float,
        "LiabilitiesAndEquity": float,
    },
    blocksize="256MB")

In [23]:
ddf_currency = dd.read_csv(
    currency_csv,
    usecols=columns_currency,
    on_bad_lines="error",
    assume_missing=True,
    dtype={
        "year": int,
        "month": int,
        "from_currency": str,
        "rate": float
    },
    blocksize="256MB")

In [25]:
#enrich the annual report with asof values from the registrations
ddf = dd_enrich_with_asof_values(
    ddf_annualreport, 
    ddf_registrations, 
    values=['Industry', 'CompanyType', 'Address', 'Status'], 
    date_col_df='PublicationDate', 
    date_col_registrations='FromDate'
    )

In [26]:
# convert currency
ddf = convert_currency(ddf, ddf_currency, 
                        amount_cols=['ProfitLoss', 'Equity', 'Assets', 'LiabilitiesAndEquity'], 
                        currency_col='Currency', 
                        date_col='PublicationDate')

In [28]:
#drop currency column
ddf = ddf.drop(columns=['Currency'])

#print list of column names
columns = ddf.columns
columns

Index(['PublicationDate', 'CVR', 'ProfitLoss', 'Assets', 'Equity',
       'LiabilitiesAndEquity', 'Industry', 'CompanyType', 'Address', 'Status'],
      dtype='object')

In [29]:
output_columns = [
    "START_DATE",
    "CVR",
    "PROFIT_LOSS",
    "ASSETS",
    "EQUTY",
    "LIABILITIES_AND_EQUITY",
    "INDUSTRY",
    "COMPANY_TYPE",
    "MUNICIPALITY",
    "STATUS"
]

In [30]:
dict(zip(columns, output_columns))

{'PublicationDate': 'START_DATE',
 'CVR': 'CVR',
 'ProfitLoss': 'PROFIT_LOSS',
 'Assets': 'ASSETS',
 'Equity': 'EQUTY',
 'LiabilitiesAndEquity': 'LIABILITIES_AND_EQUITY',
 'Industry': 'INDUSTRY',
 'CompanyType': 'COMPANY_TYPE',
 'Address': 'MUNICIPALITY',
 'Status': 'STATUS'}

In [31]:
ddf = ddf.rename(columns=dict(zip(columns, output_columns)))

In [32]:
#show head of the dataframe
ddf.head()

,START_DATE,CVR,PROFIT_LOSS,ASSETS,EQUTY,LIABILITIES_AND_EQUITY,INDUSTRY,COMPANY_TYPE,MUNICIPALITY,STATUS
0,2013-01-01,20643048,7616.0,45074.0,27149.0,45074.0,731110,APS,8260,NORMAL
1,2013-01-01,19217388,43393.0,516985.0,455341.0,516985.0,412000,APS,2150,NORMAL
2,2013-01-01,31476836,426447.0,1137021.0,614372.0,1137021.0,711290,APS,7500,NORMAL
3,2013-01-01,25512294,613702.0,4505529.0,2335109.0,4505529.0,692000,APS,8240,NORMAL
4,2013-01-01,27285627,-313745.0,3378351.0,13717.0,3378351.0,682030,APS,5250,NORMAL
